In [20]:
import pandas as pd
import numpy as np

# Imported PyTorch libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import random_split


# All images stored in CNN_Data/CNN
# Collected by Carlie

In [2]:
# Clara
# A generic resizing function that resizes images appropriately in 500 x 500
transform = transforms.Compose([
    transforms.Resize((500, 500)),
    transforms.ToTensor()
])

In [3]:
# Clara
# # Adjust dataset based upon specifications of the project.
dataset = datasets.ImageFolder('CNN_Data/CNN', transform=transform)

In [ ]:
# Clara
# Grab number of classes within the dataset based on the directory structure
num_classes = len(dataset.classes)
print(f"{num_classes} classes: {dataset.classes}")
print(f"Total images: {len(dataset)}")

Found 2 classes: ['official_razorback', 'other']
Total images: 62


In [ ]:
# Clara
# Utilizing an 80/20 training-test split, mostly because of the small dataset size.
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [ ]:
# Clara
# Create dataloaders based on the train and test datasets.
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Clara
# Print the quantities of images.
# Training is super close to 50, which is nice.
print(f"Training images: {train_size}")
print(f"Testing images: {test_size}\n")

Training images: 49
Testing images: 13



In [ ]:
# Clara
# Define baseline CNN moddel. No frills.
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        
        # Layer 1: Find basic features (edges, colors)
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        
        # Layer 2: Find more complex patterns
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        
        # Layer 3: Find even more detail
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        
        # Layer 4: Combine everything to make predictions
        self.fc1 = nn.Linear(128 * 62 * 62, 256)
        self.fc2 = nn.Linear(256, num_classes)
        
        # Dropout to prevent overfitting
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        # Pass through conv layer 1
        x = self.pool(F.relu(self.conv1(x)))  # 500->250
        
        # Pass through conv layer 2
        x = self.pool(F.relu(self.conv2(x)))  # 250->125
        
        # Pass through conv layer 3
        x = self.pool(F.relu(self.conv3(x)))  # 125->62
        
        # Flatten
        x = x.view(-1, 128 * 62 * 62)
        
        # Final classification layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

In [ ]:
# Clara
# Create the model
model = SimpleCNN(num_classes=num_classes)

In [35]:
# Clara
# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(f"Using device: {device}")

Using device: cpu


In [ ]:
# Clara
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Define the training function
def train(num_epochs=10):
    # Train CNN
    model.train()
    
    for epoch in range(num_epochs):
        total_loss = 0
        correct = 0
        total = 0
        
        for batch_num, (images, labels) in enumerate(train_loader):
            # Move data to GPU/CPU
            images = images.to(device)
            labels = labels.to(device)
            
            # Clear previous gradients
            optimizer.zero_grad()
            
            # Make predictions
            outputs = model(images)
            
            # Calculate loss
            loss = criterion(outputs, labels)
            
            # Update weights
            loss.backward()
            optimizer.step()
            
            # Track accuracy
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        # Print results for this epoch
        avg_loss = total_loss / len(train_loader)
        accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.2f}%")

In [ ]:
# Clara
# Train the model. Observe that the accuracies aren't bad.
train()

Epoch 1/10 - Train Loss: 10.2388, Train Accuracy: 55.10%
Epoch 2/10 - Train Loss: 6.7047, Train Accuracy: 48.98%
Epoch 3/10 - Train Loss: 1.1226, Train Accuracy: 53.06%
Epoch 4/10 - Train Loss: 0.6663, Train Accuracy: 65.31%
Epoch 5/10 - Train Loss: 1.2833, Train Accuracy: 55.10%
Epoch 6/10 - Train Loss: 0.8298, Train Accuracy: 65.31%
Epoch 7/10 - Train Loss: 0.5729, Train Accuracy: 71.43%
Epoch 8/10 - Train Loss: 0.5224, Train Accuracy: 63.27%
Epoch 9/10 - Train Loss: 0.4129, Train Accuracy: 75.51%
Epoch 10/10 - Train Loss: 0.3917, Train Accuracy: 87.76%


In [ ]:
# Clara
# Test model now.
def test():

    # Set to evaluation mode
    model.eval()
    
    correct = 0
    total = 0
    test_loss = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    avg_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total
    
    print("Test:")
    print(f"Test Loss: {avg_loss:.4f}")
    print(f"Test Accuracy: {accuracy:.2f}%")
    print(f"Correct: {correct}/{total}")

In [36]:
# Clara
# Test
test()

Test:
Test Loss: 0.7001
Test Accuracy: 38.46%
Correct: 5/13
